In [1]:
import xgboost
print(xgboost.__version__)

3.1.1


In [2]:
import sys , xgboost as xgb
print(sys.executable)
print(xgb.__version__)
print(xgb.__file__)

D:\ML Projects\Anaconda Projects\HouseTS\.venv\Scripts\python.exe
3.1.1
D:\ML Projects\Anaconda Projects\HouseTS\.venv\Lib\site-packages\xgboost\__init__.py


In [4]:
# Imports
import pandas as pd
import numpy as np
import optuna
import mlflow
import mlflow.xgboost
from sklearn.metrics import mean_absolute_error , mean_squared_error ,r2_score
from xgboost import XGBRegressor

D:\ML Projects\Anaconda Projects\HouseTS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_df = pd.read_csv("../Data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("../Data/processed/feature_engineered_eval.csv")

# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [7]:
# Define Optuna objective function with MLflow
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [8]:
# Run Optuna study with MLflow
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("../mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/11/02 11:51:35 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-11-02 11:51:35,192] A new study created in memory with name: no-name-2396f2b9-c6ab-46f7-af4a-0d56515d6067
[I 2025-11-02 11:51:42,550] Trial 0 finished with value: 80514.29344692096 and parameters: {'n_estimators': 373, 'max_depth': 3, 'learning_rate': 0.2717221378793368, 'subsample': 0.6659642071219676, 'colsample_bytree': 0.5958591798290669, 'min_child_weight': 7, 'gamma': 0.767143693700405, 'reg_alpha': 0.6510844279406185, 'reg_lambda': 0.8347506403083116}. Best is trial 0 with value: 80514.29344692096.
[I 2025-11-02 11:52:29,945] Trial 1 finished with value: 74166.73837070339 and parameters: {'n_estimators': 977, 'max_depth': 9, 'learning_rate': 0.03124127959163518, 'subsample': 0.8104812221812392, 'colsample_bytree': 0.9359305388620751, 'min_child_weight': 8, 'gamma': 2.1631538112561426, 'reg_alpha': 1.5329402575239752, 'reg_lambda': 9.79

Best params: {'n_estimators': 829, 'max_depth': 6, 'learning_rate': 0.028096139611649674, 'subsample': 0.6847664101536527, 'colsample_bytree': 0.6893079110916576, 'min_child_weight': 1, 'gamma': 3.789623220317366, 'reg_alpha': 0.0003490808666073794, 'reg_lambda': 7.1563085977605185}


In [9]:
# Train final model with best params and log to MLflow
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 32439.87594264014
RMSE: 71718.65330992943
R²: 0.9602511760890882


D:\ML Projects\Anaconda Projects\HouseTS\.venv\Lib\site-packages\xgboost\sklearn.py:1115: UserWarning: [11:59:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/11/02 11:59:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
